<p><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"></p>
<h1 id="-unSkript-Runbooks-">unSkript Runbooks <a class="jp-InternalAnchorLink" href="#-unSkript-Runbooks-" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success"><strong>&nbsp;This runbook adds tags to selectedAWS Resources.</strong></div>
<p>&nbsp;</p>
<center>
<h2 id="Enforce-Mandatory-Tags-Across-All-AWS-Resources">Enforce Mandatory Tags Across All AWS Resources<a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<ol>
<li>List all the Resources ARNs that do not have this tag in the given region.</li>
<li>Select teh resources to tag - with value from input parameters.</li>
<li>Add Tags to the selected AWS Resources.</li>
</ol>

Here we will use unSkript AWS Get Untagged Resources Lego. This lego take region: str as input. This inputs is used to find out all Untagged Resources.

In [167]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_resources_missing_tag_printer(output):
    if output is None:
        return
    pprint.pprint(f"there are {len(output)} resources missing tag {Tag_Key}. We can fix a max of 20." )


@beartype
def aws_get_resources_missing_tag(handle, region: str, tag:str) -> List:
    """aws_get_resources_missing_tag Returns an List of Untagged Resources.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter resources.

        :rtype: List of untagged resources.
    """

    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []

    arnKeywordsToIgnore = ["sqlworkbench",
                           "AutoScalingManagedRule",
                           "sagarProxy",
                           "fsap-0f4d1bbd83f172783",
                           "experiment"]

    try:
        response = aws_get_paginator(ec2Client, "get_resources", "ResourceTagMappingList")
        for resources in response:
            if not resources["Tags"]:
                #no tags at all!!
                arnIgnore = False
                for substring in arnKeywordsToIgnore:
                    if substring in resources["ResourceARN"]:
                        arnIgnore = True
                if not arnIgnore:
                    # instance is missing tag
                    result.append(resources["ResourceARN"])
            else:
                #has tags
                allTags = True
                keyList = []
                tagged_instance = resources['Tags']
                #print(tagged_instance)
                #get all the keys for the instance
                for kv in tagged_instance:
                    key = kv["Key"]
                    keyList.append(key)
                #see if the required tags are represented in the keylist
                #if they are not - the instance is not in compliance
                if tag not in keyList:
                    allTags = False
                if not allTags:
                    arnIgnore = False
                    for substring in arnKeywordsToIgnore:
                        if substring in resources["ResourceARN"]:
                            arnIgnore = True
                    if not arnIgnore:
                        # instance is missing tag
                        result.append(resources["ResourceARN"])

    except Exception as error:
        result.append({"error":error})

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag": "Tag_Key"
    }''')
task.configure(outputName="UntaggedResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_resources_missing_tag, lego_printer=aws_get_resources_missing_tag_printer, hdl=hdl, args=args)

In [165]:
import ipywidgets as widgets

# Maximum number of checkboxes to display
#the API has a max of 20 to update at once.
max_checkboxes = 20
checked = False
# Create checkboxes
checkboxes = [widgets.Checkbox(value=checked, description=untagged,style=dict(description_width='initial'),layout=dict(width='200%') ) for untagged in UntaggedResources[:max_checkboxes]]

# Create a VBox container to display the checkboxes vertically
checkboxes_container = widgets.VBox(checkboxes)

# Display the checkboxes
display(checkboxes_container)

# List to store the checked states
checked_list = []
firstRun = True

# Function to update the checked_list
def update_checked_list(**kwargs):
    checked_list.clear()
    checked_list.extend([untagged for untagged, value in kwargs.items() if value])
    global firstRun
    if not firstRun:
        print("Checked items:", checked_list)
    firstRun = False
    
# Create a dictionary of Checkbox widgets and their names
checkbox_dict = {untagged: checkbox for untagged, checkbox in zip(UntaggedResources, checkboxes)}

# Create the interactive_output widget
output = widgets.interactive_output(update_checked_list, checkbox_dict)

# Print the checked list initially
update_checked_list(**{name: checkbox.value for name, checkbox in checkbox_dict.items()})


# Display the output
display(output)


Here we will use unSkript AWS Attach Tags to Resources Lego. This lego take handle, resource_arn: list, tag_key: str, tag_value: str, region: str as input. This input is used to attach mandatory tags to all untagged Resources.

In [149]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def aws_attach_tags_to_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_attach_tags_to_resources(
    handle,
    resource_arn: list,
    tag_key: str,
    tag_value: str,
    region: str
    ) -> Dict:
    """aws_attach_tags_to_resources Returns an Dict of resource info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type resource_arn: list
        :param resource_arn: Resource ARNs.

        :type tag_key: str
        :param tag_key: Resource Tag Key.

        :type tag_value: str
        :param tag_value: Resource Tag value.

        :type region: str
        :param region: Region to filter resources.

        :rtype: Dict of resource info.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = {}
    try:
        response = ec2Client.tag_resources(
            ResourceARNList=resource_arn,
            Tags={tag_key: tag_value}
            )
        result = response

    except Exception as error:
        result["error"] = error

    return result



task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "Region",
    "resource_arn": "checked_list",
    "tag_key": "Tag_Key",
    "tag_value": "\\"01/01/2025\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": false,
    "iter_list_is_const": false,
    "iter_list": "checked_list",
    "iter_parameter": "resource_arn"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_tags_to_resources, lego_printer=aws_attach_tags_to_resources_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com